In [2]:
import datetime
import time
import pandas as pd

In [3]:
path_final = "./datos/ingesta_pronostico_meteo.csv"

### Conseguir data set de internet

In [4]:
url = "https://www.tutiempo.net/mexico-d-f.html?datos=por-horas&v=list"
df = pd.read_html(url)

### df

Obtener dataset de forma uniforme

In [5]:
df = df[3].T.reset_index().T.reset_index(drop=True).fillna(0)

df.columns=df[df.iloc[::,0].str.contains("Hora")].drop_duplicates().iloc[0,::]

#quitar los rows que tengan el nombre de las columnas

df = df[~df.iloc[::,0].str.contains("Hora")].reset_index(drop=True)

#poner fecha en una columna, primero sacar indices donde va cambiar la fecha

df.loc[::,"fecha"]= -1

indice = df.loc[df.Hora.str.find("-") >= 0,::].index.tolist()

funcion que agregara 0,1,2. 0 representando el primer dataset que representa la fecha dle dia de hoy,
1 la fecha del dia de mañana, y asi sucesivamente

In [6]:
contador = 0
for n in range(0,len(indice) -1 ):
    #print(n,n+1)
    df.loc[ (df.index >= indice[n]) & (df.index < indice[n + 1]) ,"fecha"] = contador
    contador = contador + 1
    if n == len(indice) - 2:
        df.loc[df.fecha == -1,"fecha"] = contador 

In [7]:
#el numero indica 0 si es fecha de hoy o 1 si es fecha del siguiente dia y asi 

from datetime import datetime, date, time, timedelta
df.loc[::,"fecha"] = df.fecha.apply(lambda x: date.today() + timedelta(days=x) )

#Quitar los rows que indicaban a que fecha pertenecia el dataset

df = df[~df.iloc[::,0].str.contains(" - ")].reset_index(drop=True)

#convertir fecha a timestand, calcular en otra columna mes y dia

df["fecha"] = (df["fecha"].astype(str) +" "+df["Hora"].astype(str)).apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M'))
df["mes"] = df["fecha"].apply(lambda fecha: fecha.month )
df["dia"] = df["fecha"].apply(lambda fecha: fecha.day )

Renombrar y seleccionar columnas 

In [8]:
df.rename(index=str, columns={"Hora": "hora", "T": "TMP","Viento":"WSP","Nubes":"RH"},inplace=True)

df = df.loc[::,["fecha","mes","dia","hora","TMP","WSP","RH"]]

#remplazar cualquier caraster por ""

df.loc[::,"TMP"] = df["TMP"].str.replace(r'\D',"")
df.loc[::,"WSP"] = df["WSP"].str.replace(r'\D',"")
df.loc[::,"RH"] = df["RH"].str.replace(r'\D',"")


In [9]:
df.loc[::,"id_station"] = "pronos" 
df = df.loc[::,["fecha","mes","dia","hora","id_station","WSP","TMP","RH"]]
df.hora = df.hora.astype(str).str.split(":").str.get(0)

In [10]:
df.head(24).to_csv(path_final,index=False)

In [11]:
df

1,fecha,mes,dia,hora,id_station,WSP,TMP,RH
0,2019-09-06 19:00:00,9,6,19,pronos,10,21,100
1,2019-09-06 20:00:00,9,6,20,pronos,8,19,100
2,2019-09-06 21:00:00,9,6,21,pronos,7,17,100
3,2019-09-06 22:00:00,9,6,22,pronos,7,17,100
4,2019-09-06 23:00:00,9,6,23,pronos,5,16,100
5,2019-09-07 00:00:00,9,7,00,pronos,4,16,100
6,2019-09-07 01:00:00,9,7,01,pronos,5,15,100
7,2019-09-07 02:00:00,9,7,02,pronos,4,15,100
8,2019-09-07 03:00:00,9,7,03,pronos,4,14,97
9,2019-09-07 04:00:00,9,7,04,pronos,4,14,92
